# TP5

## Imports

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## Chargement des données

Les fonctions pour charger les bases Movie Lens 100k et Movie Lens 1M.
On récupère un dictionnaire pour les scores et un dictionnaire pour les dates.
Le première index de ces dictionnaires est l'identifiant de l'utilisateur, et le second les films notés.

In [ ]:
def loadMovieLens(path='./data100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={} # Un dictionnaire User > Item > Rating
    times={} # Un dictionnaire User > Item > Timestamps
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

In [ ]:
def loadMovieLens1M(path='./data1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        id,title=line.split('::')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    times={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

## Représentations des données

Les matrices des scores Utilisateurs/Films sont des matrices de grandes dimensions mais sparses.
Afin de les manipuler efficacement, on emploiera 3 représentations différentes en même temps:
- Le dictionnaire des scores par utilisateurs: User > Item > Value
- Le dictionnaire des scores par films: Item > User > Value 
- La liste des triplets [User, Item, Value]

In [ ]:
# Recupère une représentation des données sous la forme triplets [user, item, value] a partir d'un dictionnaire [User > item > value]
def getCouplesUsersItems(data):
    couples = []
    for u in data.keys():
        for i in data[u].keys():
            couples.append([u,i,data[u][i]])
    return couples

# Construit le dictionnaire des utilisateurs a partir des triplets [user, item, note]
def buildUsersDict(couples):
    dicUsers = {}
    for c in couples:
        if not c[0] in dicUsers.keys():
            dicUsers[c[0]] = {}
        dicUsers[c[0]][c[1]] = float(c[2])
    return dicUsers

# Construit le dictionnaire des objets a partir des triplets [user, item, note]
def buildItemsDict(couples):
    dicItems = {}
    for c in couples:
        if not c[1] in dicItems:
            dicItems[c[1]] = {}
        dicItems[c[1]][c[0]] = float(c[2])
    return dicItems

## Données de temps

Afin d'exploiter les données temporelles, on discrétise le temps en bins et on construit un vecteur qui a chaque triplets [user, item, value] associe le bins temporel correspondant.

In [ ]:
def getTimeBins(couples, timedic, nbins):
    timestamps = np.zeros(len(couples))
    for i,c in enumerate(couples):
        timestamps[i] = timedic[c[0]][c[1]]
    time_bins = np.linspace(np.min(timestamps), np.max(timestamps), nbins+1)
    times = np.zeros(len(couples),int)
    for i in xrange(1,len(time_bins)):
        times = times + (timestamps > time_bins[i])
    return times

## Séparation des données en Train / Test

Pour pouvoir séparer les données en ensembles de Train et de Test, on utilisera la liste des triplets [User, Item, Scores].

In [ ]:
# Split l'ensemble des triplets [user, item, note] en testProp% données de test et (1 - testProp) données de train
def splitTrainTest(couples,testProp):
    perm = np.random.permutation(couples)
    splitIndex = int(testProp * len(couples))
    return perm[splitIndex:], perm[:splitIndex]

# Modèles

On implémente ici les différents modèles. Les baselines prédisent simplement la note moyenne pour un utilisateur (ou pour un film) donné. Les modèles de factorisation matricielles tentent d'approximer les valeurs connues de la matrice des scores par un produit de deux matrices de dimensions inférieurs.

## Factorisation matricielle sans biais

On calcule les deux matrices P et Q tel que pour les exemples connus, PQ ~= X, où X est la matrice des scores.
Pour prédire, il suffit alors de lire dans la matrice PQ les nouveaux exemples.

In [4]:
class matrixFactorisation():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=2000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate #Pour l'optimisation alternée: 0 si non.
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.loss = []
        #Choix du paramètre a optimisé en cas d'optimisation alternée
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                #choix d'une entrée aléatoire
                r = np.random.randint(len(couples)) 
                user = couples[r][0]
                item = couples[r][1]
                # initialisation des nouveaux vecteurs p et q
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k)
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1)
                # Descente de gradient
                tmp = dataUsers[user][item] - self.p[user].dot(self.q[item])[0][0]
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                loss = loss + tmp*tmp #(Sans le terme de régularisation)
            self.loss.append(loss)
            # Optimisation alternée
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

# Tests les données Movie Lens 100k

Les données Movie Lens 100k comprennent 100 000 scores données par 1000 utilisateurs sur 1700 films.

## Préparation des données

On extrait aléatoirement une portion (20%) des données pour constituer la base de test, et le reste sera utilisé en apprentissage.

Comme on ne souhaite ne pas évaluer les objets et les utilisateurs qui n'ont jamais été rencontré en apprentissage, on retire les couples correspondants de l'ensemble de test.

Reste ensuite à reconstruire les deux dictionnaires a partir de ces liste de couples.

In [ ]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation matricielle sans biais

Après apprentissage, la factorisation matricielle donne une erreur moyenne en test de 0.9
Il est meilleur que les baselines de 0.1 point.
On note que le loss en apprentissage est de 0.84. 
Il est possible que l'on obtienne de meilleurs score de généralisation en test en augmentant la régularisation mais au vu du score actuel en apprentissage, le gain devrait rester assez faible.

In [ ]:
model3 = matrixFactorisation(10, alternate=0)
model3.fit(trainUsers, trainItems, trainCouples)

In [ ]:
plt.figure()
plt.plot(model3.loss)
plt.show()

In [ ]:
pred = model3.predict(testCouples)
print "Erreur de test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Experiences sur les données Movie Lens 1M

Le dataset Movie Lens 1M contient 1 million d'entrées, données par 6000 utiilisateurs sur 4000 films.

Une remarque que l'on peut déjà faire est que la matrice des scores est "moins sparse" que celle issue de la base 100k.

En effet, pour la base 100k on a 100000 notes pour une matrice 1000x1700, soit un remplissage de 17% de la matrice.

Pour la base 1M, on a 1 000 000 de notes pour une matrice 6000x4000, soit 24% de remplissage.

## Préparation des données

In [ ]:
# Chargement
data, timestamps = loadMovieLens1M()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# Séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation Matricielle

Avec un score en apprentissage de 0.82 et de généralisation de 0.85, on peut estimer que le paramètre de régularisation choisi (lambda = 0.2) est raisonnable pour ce problème. La factorisation matricielle est aussi meilleure que celle de la base 100k, probablement parceque la matrice d'apprentissage est moins sparse.

In [ ]:
model8 = matrixFactorisation(10, alternate=0, maxIter=1000)
model8.fit(trainUsers, trainItems, trainCouples)

In [ ]:
plt.figure()
plt.plot(model8.loss)
plt.show()

In [ ]:
pred = model8.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Conclusion

Dans ce travail, nous avons implémenté des modèles de filtrage collaboratif que nous avons ensuite chercher à évaluer.
Si on a observé qu'un modèle simple de factorisation matricielle pouvait obtenir des résultats concluants, meilleurs qu'une baseline naïve, nous n'avons pu démontré d'intérêt pratiques des modèles avec biais.

Cependant, nous avons observés que si nos modèles sans biais avaient des scores similaires en apprentissage qu'en généralisation, ce n'était pas le cas de nos modèles avec biais qui obtiennent de bien meilleurs scores en apprentissage qui ne se traduisent pas en test. On peut en déduire que nous n'avons pas déterminé les bons hyperparamètres pour permettre une bonne généralisation, et que vraisemblablement on peut encore augmenter le score en généralisation de nos modèles avec biais.

Enfin, nous n'avons eu le temps ni d'évaluer les modèles avec biais temporel, ni l'influence de la dimension de factorisation matricielle, que l'on peut aussi voir comme une forme de régularisation.

# tSNE

In [157]:
from scipy.spatial import distance

class tSNE():
    def __init__(self,perp, nIter, lr, moment, dim=2):
        self.perp = perp # entre 5 et 50
        self.nIter = nIter
        self.lr = lr
        self.moment = moment
        self.dim = dim 
    def fit(self,data):
        nEx = np.shape(data)[0]
        # Matrice des distances de ||xi - xj||² #
        normx = np.sum(data**2,1)
        normx = np.reshape(normx, (1, nEx))
        distancex = normx + normx.T - 2 * data.dot(data.T)
        # Calcul des sigma ---------------------------------------------------------------#
        lperp = np.log2(self.perp)
        # initialisation bornes pour la recherche dichotomique #
        sup = np.ones((nEx,1)) * np.max(distancex)
        inf = np.zeros((nEx,1))
        self.sigma = (sup + inf) / 2.
        # recherche dichotomique #
        stop = False
        while not stop:
            # Calculer la matrice des p(i|j)
            self.pcond = np.exp(-distancex / (2. * (self.sigma**2)))
            self.pcond = self.pcond / np.sum(self.pcond - np.eye(nEx),1).reshape(nEx,1)
            # Calculer l'entropie de p(i|j)
            entropy = - np.sum(self.pcond * np.log2(self.pcond), 0)
            # Mise a jour des bornes
              # Si il faut augmenter sigma
            up = entropy < lperp 
            inf[up,0] = self.sigma[up,0]
              # Si il faut baisser sigma
            down = entropy > lperp 
            sup[down,0] = self.sigma[down,0]
            # Mise a jour de sigma et condition d'arrêt
            old = self.sigma
            self.sigma = ((sup + inf) / 2.)
            if np.max(np.abs(old - self.sigma)) < 1e-5:
                stop = True
                print np.exp(entropy)
                print self.sigma.T  
        #--------------------------------------------------------------------------#
        #initialiser y
        self.embeddings = np.zeros((self.nIter+2, nEx, self.dim))
        self.embeddings[1] = np.random.randn(nEx, self.dim) * 1e-4
        #--------------------------------------------------------------------------#
        # p(ij)
        self.pij = (self.pcond + self.pcond.T) / (2.*nEx)
        np.fill_diagonal(self.pij, 0)
        # Descente de Gradient
        loss = []
        for t in xrange(1,self.nIter+1):
            # Matrice des distances 
            normy = np.sum((self.embeddings[t]**2),1)
            normy = np.reshape(normy, (1, nEx))
            distancey = normy + normy.T - 2 * self.embeddings[t].dot(self.embeddings[t].T)
            # q(ij)
            # self.qij = (distancey.sum() + nEx*(nEx-1)) / (1 + distancey)
            # np.fill_diagonal(self.qij, 0)
            self.qij = 1 / (1 + distancey)
            np.fill_diagonal(self.qij, 0)
            self.qij = self.qij / self.qij.sum()
            # Descente de gradient
            yt = self.embeddings[t]
            tmpgrad = 4 * ((self.pij - self.qij) / (1 + distancey)).reshape(nEx, nEx,1)
            for i in range(nEx):
                dy = (tmpgrad[i] * (yt[i]-yt)).sum(0)
                self.embeddings[t+1][i] = yt[i] - self.lr * dy + self.moment * (yt[i] - self.embeddings[t-1,i])
            l = stats.entropy(self.pij, self.qij, 2).mean()
            loss.append(l)
            print t,l

# Digits Dataset

In [91]:
from sklearn import datasets
data = datasets.load_digits()

In [158]:
model = tSNE(30,100,1000,0)
model.fit(data.data)

[ 135.20805638  135.24193477   26.39898988 ...,  135.21719547  135.21895758
   74.20709055]
[[  4.91911469   8.38679509  11.59179135 ...,   9.254232     9.27069259
   11.59179135]]
1 4.34669798902
2 4.34669797099
3 4.34669789503
4 4.34669744698
5 4.34669444549
6 4.34667249976
7 4.34650307791
8 4.34514976509
9 4.33433744221
10 4.25878482322
11 3.9634020848
12 3.57132079063
13 3.30327843009
14 3.1143434218
15 2.97341525733
16 2.86516917814
17 2.78540389647
18 2.71804035348
19 2.66298465026
20 2.6102971879
21 2.565712242
22 2.52348673412
23 2.48658772947
24 2.44966595383
25 2.41761019938
26 2.38469527196
27 2.35609537379
28 2.32610576261
29 2.30089522368
30 2.27329662025
31 2.25123027999
32 2.22574696114
33 2.20652338116
34 2.18308084854
35 2.1661475284
36 2.14471937774
37 2.12950718104
38 2.10999510839
39 2.09597904908
40 2.07832173198
41 2.06518214324
42 2.04928621408
43 2.03682374419
44 2.02250768314
45 2.01059792263
46 1.99766893599
47 1.98624099535
48 1.97452104394
49 1.9635368272
50

In [159]:
t = np.shape(model.embeddings)[0] -1
plt.figure()
plt.plot(model.embeddings[t,:,0][data.target == 0], model.embeddings[t,:,1][data.target == 0], 'o', color="blue")
plt.plot(model.embeddings[t,:,0][data.target == 1], model.embeddings[t,:,1][data.target == 1], 'o', color="red")
plt.plot(model.embeddings[t,:,0][data.target == 2], model.embeddings[t,:,1][data.target == 2], 'o', color="cyan")
plt.plot(model.embeddings[t,:,0][data.target == 3], model.embeddings[t,:,1][data.target == 3], 'o', color="magenta")
plt.plot(model.embeddings[t,:,0][data.target == 4], model.embeddings[t,:,1][data.target == 4], 'o', color="yellow")
plt.plot(model.embeddings[t,:,0][data.target == 5], model.embeddings[t,:,1][data.target == 5], 'o', color="black")
plt.plot(model.embeddings[t,:,0][data.target == 6], model.embeddings[t,:,1][data.target == 6], 'o', color="white")
plt.plot(model.embeddings[t,:,0][data.target == 7], model.embeddings[t,:,1][data.target == 7], 'o', color=(0.5, 0.5, 0))
plt.plot(model.embeddings[t,:,0][data.target == 8], model.embeddings[t,:,1][data.target == 8], 'o', color=(0,0.5,0.5))
plt.plot(model.embeddings[t,:,0][data.target == 9], model.embeddings[t,:,1][data.target == 9], 'o', color=(0.5,0,0.5))
plt.show()

# Movie Lens 100k

In [160]:
import pickle as pkl
fichier = open("./model.p")
reco = pkl.load(fichier)
fichier.close()

In [161]:
movies = np.zeros((len(reco.q),10))
titles = []
for i,q in enumerate(reco.q.keys()):
    movies[i] = np.reshape(reco.q[q],10,1)
    titles.append(q)

In [167]:
model = tSNE(30,1000,1e3,0)
model.fit(movies)

[ 135.27612107    8.34584574  135.24308361 ...,  135.07374513  135.43369759
   31.01567488]
[[ 0.16225067  0.59890049  0.07240029 ...,  0.17880061  0.16592437
   0.59890049]]
1 2.99518240734
2 2.99518240155
3 2.9951824821
4 2.99518475245
5 2.99523693777
6 2.99631040446
7 3.00596573184
8 3.01723912562
9 3.01969133128
10 3.02422732447
11 2.99985619227
12 2.92572093204
13 2.78466134904
14 2.68342671718
15 2.62634608385
16 2.58000431357
17 2.55767132782
18 2.52400028136
19 2.512455401
20 2.48802305488
21 2.48176108074
22 2.46272369244
23 2.45824583519
24 2.44303813537
25 2.43978918814
26 2.42651758724
27 2.42430579973
28 2.41325691365
29 2.41152959559
30 2.40115036935
31 2.39998202387
32 2.39108955324
33 2.39002207189
34 2.38139737183
35 2.38081416989
36 2.37296779492
37 2.37251424621
38 2.36499584803
39 2.36482789826
40 2.35771257927
41 2.35777444548
42 2.35095436914
43 2.35132096072
44 2.34471315526
45 2.3453889152
46 2.33890875866
47 2.3398424756
48 2.33343599774
49 2.33457410825
50 2.3

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:29: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: invalid value encountered in greater


In [174]:
plt.figure()
plt.plot(model.embeddings[11,:,0],model.embeddings[11,:,1], 'o', color="blue")


#y=[2.56422, 3.77284,3.52623,3.51468,3.02199]
#z=[0.15, 0.3, 0.45, 0.6, 0.75]
#n=[58,651,393,203,123]

#fig, ax = plt.subplots()
#ax.scatter(z, y)

for i, txt in enumerate(titles):
    if (np.random.rand() > .9):
        plt.annotate(txt.decode('latin-1'), model.embeddings[11,i])

plt.show()